In [1]:
!pip install tensorboard


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.optim as optim
from torchvision import datasets ,transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


In [3]:
from PIL import Image
import os

# Define your folder path
folder_path = "/kaggle/input/butterfly-images40-species/valid/ADONIS"

# Loop through each image in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Check if it's a file and has an image extension
    if os.path.isfile(file_path) and file_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'tiff')):
        with Image.open(file_path) as img:
            width, height = img.size
            print(f"{file_name}: {width}x{height}")


5.jpg: 224x224
1.jpg: 224x224
4.jpg: 224x224
3.jpg: 224x224
2.jpg: 224x224


In [4]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize image to 128x128
    transforms.RandomHorizontalFlip(),  # Random flip for augmentation
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalization
])

valid_test_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize image to 128x128
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalization
])


Load Datasets Using ImageFolder: The ImageFolder class will automatically load the images and label them based on the folder names.

In [5]:
# Load train, validation, and test datasets
train_dataset = datasets.ImageFolder(root='/kaggle/input/butterfly-images40-species/train', transform=train_transform)
valid_dataset = datasets.ImageFolder(root='/kaggle/input/butterfly-images40-species/valid', transform=valid_test_transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/butterfly-images40-species/test', transform=valid_test_transform)

In [6]:
# Create DataLoaders for train, validation, and test datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


In [7]:
# Get a batch from the train loader
images, labels = next(iter(train_loader))

# Displaying the batch shape
print(images.shape)  # Shape will be [batch_size, channels, height, width]
print(labels.shape)  # Shape will be [batch_size]


torch.Size([32, 3, 128, 128])
torch.Size([32])


In [8]:
from torchvision import datasets

# Get the number of classes
num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")

# Print class names
print("Class names:", train_dataset.classes)


Number of classes: 100
Class names: ['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO', 'ARCIGERA FLOWER MOTH', 'ATALA', 'ATLAS MOTH', 'BANDED ORANGE HELICONIAN', 'BANDED PEACOCK', 'BANDED TIGER MOTH', 'BECKERS WHITE', 'BIRD CHERRY ERMINE MOTH', 'BLACK HAIRSTREAK', 'BLUE MORPHO', 'BLUE SPOTTED CROW', 'BROOKES BIRDWING', 'BROWN ARGUS', 'BROWN SIPROETA', 'CABBAGE WHITE', 'CAIRNS BIRDWING', 'CHALK HILL BLUE', 'CHECQUERED SKIPPER', 'CHESTNUT', 'CINNABAR MOTH', 'CLEARWING MOTH', 'CLEOPATRA', 'CLODIUS PARNASSIAN', 'CLOUDED SULPHUR', 'COMET MOTH', 'COMMON BANDED AWL', 'COMMON WOOD-NYMPH', 'COPPER TAIL', 'CRECENT', 'CRIMSON PATCH', 'DANAID EGGFLY', 'EASTERN COMA', 'EASTERN DAPPLE WHITE', 'EASTERN PINE ELFIN', 'ELBOWED PIERROT', 'EMPEROR GUM MOTH', 'GARDEN TIGER MOTH', 'GIANT LEOPARD MOTH', 'GLITTERING SAPPHIRE', 'GOLD BANDED', 'GREAT EGGFLY', 'GREAT JAY', 'GREEN CELLED CATTLEHEART', 'GREEN HAIRSTREAK', 'GREY HAIRSTREAK', 'HERCULES MOTH', 'HUMMING BIRD HAWK MOTH', 'IND

In [9]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        
        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(128 * 16 * 16, 512)  # Adjust size based on image size
        self.fc2 = nn.Linear(512, num_classes)  # Output layer

    def forward(self, x):   #x -> tensor ely hasht8l 3leha 
       # print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten for Fully Connected Layer
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation (CrossEntropyLoss includes softmax)
        
        return x
num_classes = 100
model = CustomCNN(num_classes)
print(model)


CustomCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=100, bias=True)
)


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model.to(device)


CustomCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=100, bias=True)
)

In [16]:
import torch.optim as optim

writer = SummaryWriter(log_dir="./logs")  # Creates a TensorBoard log directory

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning Rate Scheduler (Reduce LR if validation loss stops improving)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=6, factor=0.5, verbose=True)

# Early Stopping Parameters
patience = 25  # Number of epochs to wait before stopping if no improvement
best_val_loss = np.inf  # Track the best validation loss
epochs_no_improve = 0  # Count epochs without improvemen

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_loader):.4f}")
    writer.add_scalar("Loss/train", running_loss / len(train_loader), epoch)
    # Validation Step
    model.eval()
    correct = 0
    total = 0
    val_loss=0.0
    
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    accuracy = 100 * correct / total
    print(f" Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")
    writer.add_scalar("Loss/valid", val_loss, epoch)
    writer.add_scalar("Accuracy/valid", accuracy, epoch)
    
    # Reduce learning rate if validation loss stops improving
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_loss)
    new_lr = optimizer.param_groups[0]['lr']

    # Print when LR changes
    if new_lr != old_lr:
        print(f"Learning Rate changed from {old_lr:.6f} to {new_lr:.6f}")


    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pth")  # Save the best model
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation loss. Epochs without improvement: {epochs_no_improve}")


    if epochs_no_improve >= patience:
        print(f"Early stopping triggered after  {epochs_no_improve}, epochs without improvement!")
        break  # Stop training if no improvement
print("Training complete!")
writer.close()



Epoch [1/100], Training Loss: 0.0909
 Validation Loss: 1.9345, Validation Accuracy: 75.40%
Epoch [2/100], Training Loss: 0.0968
 Validation Loss: 1.6866, Validation Accuracy: 75.60%
Epoch [3/100], Training Loss: 0.0850
 Validation Loss: 1.8099, Validation Accuracy: 75.20%
No improvement in validation loss. Epochs without improvement: 1
Epoch [4/100], Training Loss: 0.0936
 Validation Loss: 1.9214, Validation Accuracy: 74.60%
No improvement in validation loss. Epochs without improvement: 2
Epoch [5/100], Training Loss: 0.0705
 Validation Loss: 2.0492, Validation Accuracy: 77.20%
No improvement in validation loss. Epochs without improvement: 3
Epoch [6/100], Training Loss: 0.0877
 Validation Loss: 1.7705, Validation Accuracy: 75.80%
No improvement in validation loss. Epochs without improvement: 4
Epoch [7/100], Training Loss: 0.0771
 Validation Loss: 2.3182, Validation Accuracy: 72.20%
No improvement in validation loss. Epochs without improvement: 5
Epoch [8/100], Training Loss: 0.0785
 

In [17]:
# Testing loop
model.eval()  # Set the model to evaluation mode
test_correct = 0
test_total = 0

with torch.no_grad():  # No gradient computation during testing
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = test_correct / test_total * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 82.00%


In [13]:
!zip -r logs.zip ./logs


  adding: logs/ (stored 0%)
  adding: logs/events.out.tfevents.1739154542.fe2d06c5b425.31.0 (deflated 63%)


In [14]:
!ls -R ./logs


./logs:
events.out.tfevents.1739154542.fe2d06c5b425.31.0


###RUNLOCALLY with command
C:\Users\username\AppData\Roaming\Python\Python312\Scripts\tensorboard.exe --logdir=logs

then open http://localhost:6006/ in your browser.

